In [3]:
# 🔒 W&B 완전 비활성화 (로그인/키 요청 방지)
!pip -q uninstall -y wandb

import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"


In [4]:
# ================================
# LoRA fine-tuning (Korean SFT) — Response-only loss, robust to Transformers versions
# ================================
!pip -q install -U transformers peft accelerate torch

import os, random
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, set_seed
from peft import LoraConfig, get_peft_model, TaskType

# (중복 방어) W&B 비활성화
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

# 1) 기본 설정
SEED = 42
set_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 2) 모델/토크나이저
# ── 한국어 품질 권장: KoGPT2
BASE_MODEL = "skt/kogpt2-base-v2"
# 필요 시 distilgpt2로 바꾸실 수 있어요(한국어 성능은 낮을 수 있음):
# BASE_MODEL = "distilgpt2"

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
    tok.pad_token_id = tok.eos_token_id

# 템플릿 태그(토큰 경계 안정)
INST_TAG = "### 지시문:"
RESP_TAG = "### 응답:"
tok.add_special_tokens({"additional_special_tokens":[INST_TAG, RESP_TAG]})

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
model.resize_token_embeddings(len(tok))
model.to(device)

# 3) LoRA 설정 (GPT-2 계열 핵심 선형층만)
#   - c_attn : Q/K/VI 합성 선형층
#   - c_proj : 어텐션/MLP 출력 선형층
#   - c_fc   : MLP 확장 선형층(mlp.c_fc를 넓게 매칭)
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["c_attn","c_proj","c_fc"]
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# 4) 다양한 한국어 지시문 데이터(작지만 폭넓게)
def ex(prompt, answer):
    return f"{INST_TAG}\n{prompt}\n\n{RESP_TAG}\n{answer}{tok.eos_token}"

pairs = [
    # 요약/한줄요약
    ("한 줄 요약: 연준 발표 이후 주식시장이 상승했다.", "연준 발표 이후 주가가 상승했다."),
    ("한 줄 요약(이모지 금지, 25자 이내): 비가 오고 교통 혼잡이 심해졌다.", "비로 인해 교통 혼잡이 심해졌다."),
    # 번역
    ("영어로 번역: 정말 감사합니다.", "Thank you very much."),
    ("영어로 번역: 사과는 건강에 좋다.", "Apples are good for health."),
    ("한국어로 번역: Keep it concise and clear.", "간결하고 명확하게 작성하세요."),
    # 형식/톤 변환
    ("메일 첫 문장(공손체): 회의 일정 조율을 정중히 요청", "안녕하세요. 회의 일정 관련하여 가능한 시간을 여쭙고자 연락드립니다."),
    ("문장을 더 공손하게: 내일까지 자료 주세요.", "번거로우시겠지만 내일까지 자료를 공유해 주실 수 있을까요?"),
    ("반말로 바꾸기: 오늘 일정 확인 부탁드립니다.", "오늘 일정 확인해줘."),
    # 키워드/추출
    ("키워드 3개 추출: 인공지능이 의료 영상 판독을 보조해 정확도를 높였다.", "인공지능, 의료 영상, 정확도"),
    ("해시태그 3개 생성: 여름철 수분 섭취의 중요성", "#여름건강 #수분섭취 #열사병예방"),
    # 구조화(JSON)
    ("JSON으로 요약(키: title, sentiment): '서비스 개선 공지에 고객 반응이 대체로 긍정적이다.'",
     '{"title":"서비스 개선 공지 반응","sentiment":"positive"}'),
    ("항목별 요약(JSON: pros, cons): '배터리 오래가나 무게가 조금 무겁다.'",
     '{"pros":["배터리 오래감"],"cons":["무게가 다소 무거움"]}'),
    # 규칙/제약
    ("규칙: 한 문장, 마침표로 끝내기 — 딥러닝을 정의해줘.", "딥러닝은 다층 신경망으로 복잡한 패턴을 학습하는 기계학습 기법이다."),
    ("불릿 3개로 정리: 집중력을 높이는 방법", "• 방해 요소 제거\n• 짧은 목표 설정\n• 일정한 휴식"),
    # 설명/정의
    ("한 문장 정의: 머신러닝.", "머신러닝은 데이터에서 패턴을 학습해 예측을 수행하는 기술이다."),
    ("초보자에게 설명: 과적합이 뭐야?", "과적합은 학습 데이터에만 지나치게 맞춰져 새로운 데이터에서 성능이 떨어지는 현상이다."),
    # 분류/판정(간단)
    ("감정 분류(긍/부정 중 하나): '이 제품 정말 마음에 든다.'", "긍정"),
    ("감정 분류(긍/부정 중 하나): '배송이 너무 느려서 실망했다.'", "부정"),
    # 변환/요청
    ("문장을 더 간결하게: 본 건과 관련하여 검토 결과를 전달드립니다.", "관련 검토 결과를 전달드립니다."),
    ("숫자만 추출: 주문번호 A-001-39", "00139"),
    # 일정/비즈니스
    ("회의 아젠다 3개 제안(한 줄식): 온라인 세미나 준비 회의", "목표 확인\n역할 분담\n타임라인 확정"),
    ("납기 연장 요청 메일 첫 문장(공손체, 한 문장):", "안녕하세요, 납기 일정 관련하여 부득이하게 연장을 요청드리고자 연락드립니다."),
    # 스타일/재구성
    ("문장을 더 명확하게: 데이터 처리 속도가 부족하다.", "데이터 처리 속도가 느려 성능 저하가 발생한다."),
    ("두 문장을 한 문장으로: 모델은 정확하지만 느리다. 배치 크기를 줄였다.", "정확하지만 느린 모델을 개선하기 위해 배치 크기를 줄였다."),
]

texts = []
REPEAT = 20   # 데이터가 늘면 줄이세요(10~20 권장)
for _ in range(REPEAT):
    for p, a in pairs:
        texts.append(ex(p, a))

# 5) 커스텀 데이터셋 (토큰화는 collator에서 배치로 처리)
class TextOnlyDS(Dataset):
    def __init__(self, texts): self.texts = texts
    def __len__(self): return len(self.texts)
    def __getitem__(self, i): return {"text": self.texts[i]}

split = int(len(texts)*0.9)
train_ds = TextOnlyDS(texts[:split])
val_ds   = TextOnlyDS(texts[split:])

# 6) Collator: 배치 토큰화 + "응답만 로스" 마스킹 + 안전 패딩
RESP_TAG_WITH_NL = RESP_TAG + "\n"
resp_ids = tok(RESP_TAG_WITH_NL, add_special_tokens=False)["input_ids"]

def find_subseq(seq, sub):
    L, l = len(seq), len(sub)
    for i in range(L - l + 1):
        if seq[i:i+l] == sub:
            return i
    return -1

def response_only_collate(features, tokenizer=tok, max_length=384):
    # 텍스트 배치 토큰화
    texts = [f["text"] for f in features]
    enc = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    input_ids = enc["input_ids"]
    attn = enc["attention_mask"]

    # labels 초기화(-100), "응답:" 이후만 정답 복사
    labels = torch.full_like(input_ids, -100)
    B = input_ids.size(0)
    for i in range(B):
        ids = input_ids[i].tolist()
        pos = find_subseq(ids, resp_ids)
        if pos != -1:
            start = pos + len(resp_ids)
            seq_len = int(attn[i].sum().item())  # pad 제외
            start = min(start, seq_len)
            labels[i, start:seq_len] = input_ids[i, start:seq_len]

    return {"input_ids": input_ids, "attention_mask": attn, "labels": labels}

def collate(batch):
    return response_only_collate(batch)

# 7) 학습 인자 (버전 호환: 최신/구버전 자동 fallback)
model.config.use_cache = False  # Trainer 경고 방지
def make_args():
    # 최신 버전용
    try:
        return TrainingArguments(
            output_dir="./out_lora",
            overwrite_output_dir=True,
            num_train_epochs=4,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=2,
            learning_rate=1e-4,
            lr_scheduler_type="cosine",
            weight_decay=0.0,
            logging_steps=20,
            evaluation_strategy="steps",  # ← 일부 구버전에서 미지원
            eval_steps=100,
            save_strategy="no",
            remove_unused_columns=False,
            report_to="none",
            bf16=(torch.cuda.is_available() and getattr(torch.cuda, "is_bf16_supported", lambda: False)()),
            fp16=(torch.cuda.is_available() and not getattr(torch.cuda, "is_bf16_supported", lambda: False)()),
        )
    except TypeError:
        # 구버전 호환(문제되는 인자 제거)
        return TrainingArguments(
            output_dir="./out_lora",
            overwrite_output_dir=True,
            num_train_epochs=4,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=2,
            learning_rate=1e-4,
            logging_steps=20,
            remove_unused_columns=False,
            # 아래 인자들은 구버전에서 없을 수 있어 생략
            # lr_scheduler_type, save_strategy, report_to, evaluation_strategy, eval_steps, bf16/fp16 등
        )

args = make_args()

trainer = Trainer(
    model=model, args=args,
    train_dataset=train_ds, eval_dataset=val_ds,
    data_collator=collate, tokenizer=tok
)

print("학습 시작...")
trainer.train()
print("학습 종료.")

# 8) 어댑터 저장
save_dir = "./lora_adapter_ko_sft"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tok.save_pretrained(save_dir)
print("저장:", save_dir)

# 9) 추론 함수(학습 템플릿과 동일하게!)
@torch.inference_mode()
def generate_ko(prompt, max_new_tokens=80, temperature=0.7, top_p=0.9, do_sample=False):
    model.eval()
    prefix = f"{INST_TAG}\n{prompt}\n\n{RESP_TAG}\n"
    inputs = tok(prefix, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample, temperature=temperature, top_p=top_p,
        pad_token_id=tok.eos_token_id, eos_token_id=tok.eos_token_id
    )
    txt = tok.decode(out[0], skip_special_tokens=False)
    return txt.split(RESP_TAG, 1)[-1].strip() if RESP_TAG in txt else txt.strip()

print("\n=== 데모 ===")
tests = [
    # 학습 분포와 유사(일반화 체크)
    "한 줄 요약: 배터리 수명이 길어 사용자 만족도가 높아졌다.",
    "영어로 번역: 도움 주셔서 감사합니다.",
    "메일 첫 문장(공손체): 회의 링크 재전송 요청",
    "키워드 3개 추출: 신규 기능 출시로 사용자 유입이 증가했다.",
    "JSON으로 요약(키: title, sentiment): '배송 지연 이슈가 사라져 평점이 올랐다.'",
    # 살짝 새로운 요청(전이 성능 체크)
    "두 문장 요약(30자 이내): 서버 과부하로 장애 발생, 임시 확장 적용.",
    "불릿 3개: 업무 집중력을 높이는 실천 팁",
    "문장을 더 부드럽게: 일정 변경 요청드립니다.",
    "간단 정의(한 문장): 과소적합.",
    "감정 분류(긍/부정): '포장이 엉망이라 실망스러웠다.'",
]
for p in tests:
    print("Q:", p)
    print("A:", generate_ko(p))
    print("-"*40)


Device: cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


trainable params: 1,179,648 || all params: 126,345,984 || trainable%: 0.9337


/tmp/ipython-input-45762981.py:191: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 51200, 'bos_token_id': 51200, 'pad_token_id': 51200}.


학습 시작...


...

Step,Training Loss
20,4.494700
40,3.566000
60,3.081300
80,2.483600
100,1.927000
120,1.733000
140,1.514700
160,1.325500
180,1.271600
200,1.306300


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


학습 종료.


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


저장: ./lora_adapter_ko_sft

=== 데모 ===
Q: 한 줄 요약: 배터리 수명이 길어 사용자 만족도가 높아졌다.
A: 배터리 수명 연장
간결하고 명확하게 성능이 개선됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고 명확하게 성능이 향상됐다.
간결하고
----------------------------------------
Q: 영어로 번역: 도움 주셔서 감사합니다.
A: Thank you very much.Apples are good for health.Apples are good for health.Apples are good for health.Apples are good for health.Apples are good for health
----------------------------------------
Q: 메일 첫 문장(공손체): 회의 링크 재전송 요청
A: 안녕하세요, 회의 링크 재전송 요청드립니다. 회의 링크 재전송 요청드립니다. 회의 링크 재전송 요청드립니다. 회의 링크 재전송 요청드립니다. 회의 링크 재전송 요청드립니다. 회의 링크 재전송 요청드립니다. 회의 링크 재전송 요청드립니다. 회의 링크 재전
----------------------------------------
Q: 키워드 3개 추출: 신규 기능 출시로 사용자 유입이 증가했다.
A: • 신규 기능 출시로 사용자 유입이 증가했다.• 새로운 기능 출시로 사용자 유입이 증가했다.• 새로운 기능 출시로 사용자 유입이 증가했다.• 새로운 기능 출시로 사용자 유입이 증가했다.• 새로운 기능 출시로 사용자 유입이 증가했다.• 새로운 기능 출시로 사용자 유입이 증가했다.• 새로운 기능 출시로 사용자 유입이 증가했다.• 새로운 기능 출시로 사용자 유입이 증가했